In [1]:
%matplotlib inline
# notebook
import matplotlib.pylab as pylab
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np
import pandas as pd
import networkx as nx
from math import sin, cos, sqrt, atan2, radians
from utils import *
from data import *
from trophics import *
from collections import defaultdict
#make the graphs bigger
pylab.rcParams['figure.figsize'] = (32.0, 24.0)
pylab.rcParams['font.size'] = 24

In [2]:
rawEiggData = validatedEiggData()
rawEiggData.head()

C:\Users\davie\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2862: DtypeWarning: Columns (9,22) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Recorder,Latitude (WGS84),Longitude (WGS84),Start date year,End date year,Scientific name,Common name
0,unknown,56.902729,-6.15936,1905.0,NaN,meloe violaceus,Violet Oil-beetle
1,Frank Balfour-Browne,56.899200,-6.19759,1910.0,1910.0,rhantus suturellus,NaN
2,Frank Balfour-Browne,56.899200,-6.19759,1910.0,1910.0,gyrinus substriatus,NaN
3,Frank Balfour-Browne,56.899200,-6.19759,1910.0,1910.0,enochrus fuscipennis,NaN
4,Frank Balfour-Browne,56.899200,-6.19759,1910.0,1910.0,boreonectes multilineatus,NaN


In [3]:
foodWeb = retrieveCollatedFoodWeb()
len(foodWeb)

C:\Users\davie\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DtypeWarning: Columns (2,8,12,13,22,26,27,41,44) have mixed types. Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.
C:\Users\davie\Desktop\Masters\Dissertation\Code\DissertationCode\Eigg\trophics.py:9: DtypeWarning: Columns (9,22) have mixed types. Specify dtype option on import or set low_memory=False.
  dataSetFunctions = [readFreshwaterData(), read2018GlobalDatabaseData(), readSantaBarbaraMatrix(), readSorensenData(), readJanesData()]


1886


4381

In [24]:
overall = set()

for item in foodWeb:
    item = item.split(" ")
    item = " ".join(item[:2])
    overall.add(item)

for fw in foodWeb:
    v = foodWeb[fw]
    for item in v:
        item = item.split(" ")
        item = " ".join(item[:2])
        overall.add(item)

print(len(overall))

5588


In [25]:
eiggSpecies = rawEiggData['Scientific name'].str.lower()
eiggSpecies = set(eiggSpecies)

animalsTotal = len(eiggSpecies)
totalFound = 0
for item in eiggSpecies:
    if item in overall:
        totalFound += 1
    

print(totalFound / animalsTotal)

0.09180432020330369
